To use this notebook, install the necessary packages by running the following command:  
pip install numpy matplotlib nnfs

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import nnfs 
from nnfs.datasets import spiral_data 

In [2]:
nnfs.init()

In [3]:
# Dense layer
class Layer_Dense:
    # Layer initialization
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
        
    # Forward pass
    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases

In [4]:
# ReLU activation layer
class Activation_ReLU:
    # Forward pass
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)

In [5]:
# Softmax activation layer
class Activation_Softmax:
    # Forward pass
    def forward(self, input):
        # Get unnormalized probabilities
        exp_values = np.exp(input - np.max(input, axis=1, keepdims=True))
        
        # Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        
        self.output = probabilities

In [6]:
# Create dataset
X, y = spiral_data(samples=100, classes=3)

In [7]:
# Create Dense layer with 2 input features and 3 output values 
dense1 = Layer_Dense(2, 3)
print(dense1.weights)
print(' ')

# Create ReLU activation (to be used with dense layer)
activation1 = Activation_ReLU()

# Create second Dense layer with 3 input features (as we take output 
# of previous layer here) and 3 output values (output values) 
dense2 = Layer_Dense(3, 3)
print(dense2.weights)

# Create Softmax activation (to be used with Dense layer)
activation2 = Activation_Softmax()

[[-0.01306527  0.01658131 -0.00118164]
 [-0.00680178  0.00666383 -0.0046072 ]]
 
[[-0.01334258 -0.01346717  0.00693773]
 [-0.00159573 -0.00133702  0.01077744]
 [-0.01126826 -0.00730678 -0.0038488 ]]


In [8]:
# Make a forward pass of our training data through this layer
dense1.forward(X)

# Make a forward pass through activation function 
# it takes the output of first dense layer here 
activation1.forward(dense1.output)

# Make a forward pass through second Dense layer 
# it takes outputs of activation function of first layer as inputs 
dense2.forward(activation1.output)

# Make a forward pass through activation function 
# it takes the output of second dense layer here 
activation2.forward(dense2.output)

print(activation2.output[:5])  # Print first 5 outputs

[[0.33333334 0.33333334 0.33333334]
 [0.3333332  0.3333332  0.33333364]
 [0.3333329  0.33333293 0.3333342 ]
 [0.3333326  0.33333263 0.33333477]
 [0.33333233 0.3333324  0.33333528]]
